In [1]:
import os
import argparse
import numpy as np
from itkwidgets import view
import SimpleITK as sitk
import pymskt as mskt
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import json
import pandas as pd
import datetime
import time
import torch
from pymskt.mesh.meshTransform import get_linear_transform_matrix
import sys
import gc
import pyvista as pv
import vtk
import ast
import pickle
from NSM.models.triplanar import TriplanarDecoder
from NSM.reconstruct import reconstruct_mesh
from NSM.mesh.interpolate import interpolate_points, interpolate_mesh, interpolate_common
from NSM.mesh import create_mesh
from pymskt.mesh import BoneMesh, CartilageMesh, meshTools
sys.path.append('/dataNAS/people/kenziew/projects/knee_pipeline/DEPENDENCIES/nsosim')
from nsosim.nsm_fitting import apply_transform, undo_transform
from nsosim.utils import load_model
from scipy.spatial import cKDTree
# from vtk.util.numpy_support import numpy_to_vtk

[KeOps] Compiling cuda jit compiler engine ... 
[KeOps] Warning : There were warnings or errors :
/usr/bin/ld: cannot find -lnvrtc: No such file or directory
collect2: error: ld returned 1 exit status

OK
[pyKeOps] Compiling nvrtc binder for python ... 
[KeOps] Warning : There were warnings or errors :
/usr/bin/ld: cannot find -lnvrtc: No such file or directory
collect2: error: ld returned 1 exit status

OK


In [2]:
def masked_healthy_mesh(
    mesh,
    threshold,
    percent_scalar='percent_people_with_cartilage', # Percent of people with cartilage at each point
    percent_out='percent_people_with_cartilage_masked',
    label_in='most_likely_label', # Subregion names
    label_out= 'most_likely_label_masked',  # this is the updated version "label in"
    prob_in='most_likely_prob', # what update_sub_labels expects
    prob_out= 'most_likely_prob_masked',
    keep_only= ['percent_people_with_cartilage', ],  # None = keep everything
    subset=False # True -> drop < thr points via threshold
):
    
    """
    Make a thresholded copy of `mesh` where:
      - labels outside the mask are set to 0
      - probs outside the mask are set to 0
      - percent scalar also masked (0 outside)
    Threshold can be fraction (0.8) or percent (80).
    If subset=True, returns a mesh containing only vertices/cells >= threshold.
    """
    thr = threshold / 100.0 if threshold > 1.0 else float(threshold)

    mesh2 = mesh.copy(deep=True)
    pct = np.asarray(mesh2.point_data[percent_scalar], dtype=float)
    mask = pct >= thr

    labels = np.asarray(mesh2.point_data[label_in], dtype=np.int32)
    probs  = np.asarray(mesh2.point_data[prob_in], dtype=float)

    # Masked outputs
    mesh2.point_data[label_out]   = np.where(mask, labels, 0).astype(np.int32)
    mesh2.point_data[prob_out]    = np.where(mask, probs,  0).astype(np.float32)
    mesh2.point_data[percent_out] = np.where(mask, pct,    0).astype(np.float32)

    # Optionally reduce to only >= threshold vertices/cells
    if subset:
        try:
            # Use the percent scalar for thresholding (point preference)
            mesh2 = mesh2.threshold(thr, scalars=percent_scalar, preference='point')
        except Exception:
            # Fallback: if threshold fails (e.g., mesh type), just keep masked arrays
            pass

    # Trim extra arrays if requested
    if keep_only is not None:
        keep = set(keep_only) | {label_out, prob_out, percent_out, percent_scalar}
        data = {k: mesh2.point_data[k].copy() for k in list(mesh2.point_data.keys()) if k in keep}
        # mesh2.point_data.clear()
        for k, v in data.items():
            mesh2.point_data[k] = v

    return mesh2

# Read in healthy NSM models

In [3]:
main_path = f'/dataNAS/people/anoopai/KneePipeline/mean_data/mean_cartSubregions_OAI'
final_mesh = f'mean_healthy_femur_505MOAKS0.vtk'
mean_mesh_file_name = os.path.join(main_path, final_mesh)
femur_mesh = BoneMesh(mean_mesh_file_name)

# scalars= femur_mesh.point_data.keys()

# scalar_name = 'most_likely_prob'

# dilated_mesh = femur_mesh.copy()
# dilated_mesh.point_data[f'{scalar_name}_dilated'] = femur_mesh.point_data[scalar_name]  # Replace 'my_scalar' with your scalar's name

# # Perform a smoothing operation to "dilate"
# dilated_mesh = dilated_mesh.smooth(n_iter=100)

In [19]:
femur_mesh_threshold= masked_healthy_mesh(
    mesh= femur_mesh,
    threshold = 90,
    )

In [20]:
view(geometries=[femur_mesh_threshold])

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

In [29]:
import pyvista as pv

# Assuming femur_mesh is already defined and loaded with scalars
femur_mesh = BoneMesh(mean_mesh_file_name)

# Get unique regions from the scalar field (replace 'region_scalar' with your actual scalar name)
region_scalar_name = 'most_likely_label'  # Change this to your scalar name
unique_regions = femur_mesh.point_data[region_scalar_name]

# Get unique region values
regions_of_interest = np.unique(np.array(unique_regions))

area_results = {}

# Iterate over each unique region
for region in regions_of_interest:

    # Create a copy of the mesh
    current_mesh = femur_mesh.copy()
    
    # Create a boolean mask to keep only cells that belong to the current region
    mask = current_mesh.point_data[region_scalar_name] == region
    
    # Delete all points/cells that do not belong to this region
    current_mesh = current_mesh.extract_surface().threshold(0) # Extracts surface
    
    # Use the boolean mask to keep only the cells that belong to the current region
    current_mesh = current_mesh.extract_cells(mask)
    
    # Compute the surface area of the remaining mesh
    surface_area = current_mesh.area
    
    # Store the results
    area_results[region] = surface_area

# Print out the surface area results for each region
for region, area in area_results.items():
    print(f"Surface area of region {region}: {area}")

Surface area of region 0: 1.4971924375270356
Surface area of region 11: 0.3894294712196732
Surface area of region 12: 0.09654563015450959
Surface area of region 13: 0.10421438401247252
Surface area of region 14: 0.17876906430755318
Surface area of region 15: 0.17236678432503916
